# Data Science Capstone Project - Week 3 Assignment

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download Dependencies</a>

2. <a href="#item2">Scrape and Parse Table</a>

3. <a href="#item3">Answer to Question 3 (First Submission)</a>

4. <a href="#item4">Gather Latitude and Longitude</a>

</font>
</div>

<a id='item1'></a>

## Download Dependencies

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis


In [3]:
# Install Beautiful Soup
#from bs4 import BeautifulSoup

## Beautiful Soup not used - opted to use pandas read instead

In [ ]:
## Beautiful Soup not used - opted to use pandas read instead
#Get Toronto Data: Postal Codes starting with 'M'
#import urllib2 #enable ability to read URLs
#from urllib.request import urlopen #enable ability to read URLs

#postal_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#page = urlopen(postal_url).read()
#soup = BeautifulSoup(page, 'html.parser')

#postal_frame = []
#postal_table =  soup.find('table', {'class': 'wikitable sortable'})

#rows = postal_table.findAll('tr')
#for row in rows:
#    cols=row.findAll('td')
#    cols=[ele.text.strip() for ele in cols]
#    postal_frame.append([ele for ele in cols if ele])

##postal_frame.shape
##postal_table

<a id='item2'></a>

## Scrape and Parse Table using pandas

In [4]:
#Get Toronto Data: Postal Codes starting with 'M' using pandas.read
#import urllib2 #enable ability to read URLs

import requests
postal_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(postal_url)

postal_frame = pd.read_html(page.content,attrs = {'class': 'wikitable sortable'})[0]


postal_frame.shape


(288, 3)

In [5]:
postal_frame.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
# Process dataframe
#Remove Borough 'Not assigned' rows
postal_frame_clean = postal_frame.drop(postal_frame[postal_frame.Borough == 'Not assigned'].index)

#Change Neighborhood to match Borough if Neighborhood = 'Not assigned'
postal_frame_clean['Neighbourhood'][postal_frame_clean['Neighbourhood'] == 'Not assigned'] = postal_frame_clean['Borough']

#Combine multiple neighborhoods from the same Burough into one row
postal_frame_clean = postal_frame_clean.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()

#postal_frame_clean.head()

<a id='item3'></a>

## Answer to Question 3 (First Submission)

In [7]:

postal_frame_clean.shape


(103, 3)

<a id='item4'></a>

## Gather Latitude and Longitude for each Postal code in our dataframe

In [ ]:
import geocoder # import geocoder
postal_code = 'M5G'
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [8]:
latitude

NameError: name 'latitude' is not defined

In [ ]:
longitude

## Explore DataNeighbourhood

## Segment Data

## Cluster Data